# ***CSI4180HW3***


---
Gregory Sylvester |
3/28/2024 |
Oakland University


In [ ]:
!pip install accelerate>=0.21.0

In [ ]:
!pip install accelerate -U

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install datasets
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00


In [ ]:
#homework 03 nlp 4180
#Gregory Sylvester
#3/25/2024
import numpy as np
from datasets import list_datasets, load_dataset
from pprint import pprint
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW, TrainingArguments, Trainer, AutoModel, DistilBertTokenizerFast, pipeline, TrainingArguments
import torch
from datasets import load_dataset
#import evaluate
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from transformers import DataCollatorWithPadding
import evaluate
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

# **2.1 Dataset**
--------
For this assignment, we will return to the HuggingFace datasets hub to easily
access an existing dataset. For this assignment, you may choose any of the
following tasks:
- Emotion classification
- Hate Speech Detection (warning: this requires working with and reading
potentially offensive texts)
- Hyperpartisan News Detection
- SMS Spam Detection
- Wikipedia toxic comment detection

or another dataset of your choice with approval of Prof. Wilson (just
ask on Discord). Any dataset you propose must have train and test splits
(i.e., you cannot use an evaluation-only benchmark), should be reasonably
difficult for models to classify, and should have some practical importance.
No sentiment analysis datasets for this assignment unless they have some
especially interesting feature to them (e.g., texts are multilingual or from
a unique source). You may also propose to work on another task such as
question answering or machine translation – just keep in mind that this
may add some extra (but manageable) challenges.

In [ ]:
datasets = list_datasets()
sms_spam = list_datasets(with_details=True)[datasets.index('sms_spam')]

<ipython-input-8-fec33ca76940>:1: FutureWarning: list_datasets is deprecated and will be removed in the next major version of datasets. Use 'huggingface_hub.list_datasets' instead.
  datasets = list_datasets()
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
#spliting Dataset
dataset = load_dataset('sms_spam')


Generating train split:   0%|          | 0/5574 [00:00<?, ? examples/s]

# **2.2 Fine-tuning pre-trained models**

---


After you have your dataset, select (at least) two different pre-trained models,
such as BERT & friends like RoBERTa, DistilBERT, DeBERTa, or T5, Electra,
or any others you find. The models should not be already fine-tuned on your
task (i.e., you should not use a BERT model already fine-tuned for the Go
Emotion dataset if you have selected that dataset as your target task), but
could be fine-tuned on other tasks – just verify that there is no overlap between
your task and the other task that the model has been fine-tuned on. If you
are using free-tier Colab notebooks, keep in mind the memory limitations and
avoid models that are billions of parameters (BERT-base-sized models should
be doable). You will also typically find “distilled”, “small”, or “tiny” versions
of the models which are not as memory intensive and can still be used here.
Then, fine-tune your models on the training split of your dataset. If there is no
predefined training split (or all the entire dataset is in the “train” split), select
80% of your dataset to use for training and leave the rest for testing. Then, use
your fine-tuned model to predict the labels for the data in the test split and
report the appropriate evaluation metrics. Most tasks will have a recommended
evaluation metric that is typically used and you should use the same one.


**Most of this comes from the https://huggingface.co/docs/transformers/en/tasks/sequence_classification**


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
def preprocess_function(examples):
    return tokenizer(examples["sms"], truncation=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [17]:
accuracy = evaluate.load("accuracy")
id2label = {0: "Not Spam", 1: "Spam"}
label2id = {"Not Spam": 0, "Spam": 1}

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= tokenized_dataset['train'],
    eval_dataset= tokenized_dataset['train'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.018833,0.995515
2,0.056700,0.011046,0.997309


TrainOutput(global_step=698, training_loss=0.04652482800633996, metrics={'train_runtime': 4733.3663, 'train_samples_per_second': 2.355, 'train_steps_per_second': 0.147, 'total_flos': 183930983033856.0, 'train_loss': 0.04652482800633996, 'epoch': 2.0})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/electra-small-discriminator")
model2 = AutoModelForSequenceClassification.from_pretrained(
    "google/electra-small-discriminator", num_labels=2, id2label=id2label, label2id=label2id
)

def preprocess_function(examples):
    return tokenizer(examples["sms"], truncation=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model2,
    args=training_args,
    train_dataset= tokenized_dataset['train'],
    eval_dataset= tokenized_dataset['train'], # this is the line with the error, when i do data splits and input a different tokenized set it dumps error
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/5574 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.047567,0.992465
2,0.153300,0.037736,0.993900


Checkpoint destination directory my_awesome_model/checkpoint-349 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory my_awesome_model/checkpoint-698 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=698, training_loss=0.12273661933177522, metrics={'train_runtime': 1318.42, 'train_samples_per_second': 8.456, 'train_steps_per_second': 0.529, 'total_flos': 40849171295232.0, 'train_loss': 0.12273661933177522, 'epoch': 2.0})

# **2.3 Zero-shot classification**

---


Models like GPT-2 and successors have shown that zero-shot (no training data)
text classification can be a viable alternative and require no additional finetuning. However, a new task has emerged: prompt engineering, in which practitioners need to decide on the correct way to “prompt” the model in order to
get it to solve the task. For example, researchers found that adding the line
“let’s think step-by-step” before asking a language model to solve a reasoning
problem substantially increased the model’s performance on the task.
For this part of the assignment, you will use a pretrained language model
and act as a prompt engineer: using some examples from the training data, try
to craft a prompt the causes the model to provide a label as output, e.g., using
the string “Is this text offensive? answer yes or no.” preceded by the training
2
example’s text. Try several variations to see what works well on your training
data, but only use the best prompt for your test dataset.
For any model that you can download and run yourself, you can also directly
access the output probabilities for the next token. So if you give the prompt
“Is this text offensive? answer yes or no.”, you don’t actually need to see what
the model generates. Instead, you can check P(yes|prompt) and P(no|prompt)
to see which is higher, and consider that the selected label.
You should select 2 different models and try to identify the best way to
prompt each one. You can find a large number of text generation models on
the huggingface model hub, but keep in mind the memory requirements might
be higher than what your hardware (or free-tier Colab) supports, so check the
number of parameters and any details about the RAM/vRAM requirements on
the model page. You may also be interested in highly optimized local models
like Llama.cpp which supports a wide range of models and is meant to be run on
a laptop. Another option you can choose here would be a Cloud API-based LLM
like GPT-4 or Google Gemini, but keep in mind that these are not free.1 Do not
try to use the chat interface for these, otherwise you will need to copy and paste
every single item from your test set, get the reply, start a new chat session, and
repeat, and this does not follow the best practices of how we actually use LLMs
via code. Instead, you will need to read about and use the APIs for these models
if you choose to use them. Finally, a fourth option would be to use a natural
language inference model to do zero-shot classification as described here.
After you run your zero-shot models your test data, compute the same evaluation metrics from the previous step (fine-tuning) so you can directly compare
the results. Keep in mind that while these models may not perform as well
(though they very well might!), you did not need to do any additional training
in order to get the results.

In [ ]:
pipe = pipeline("zero-shot-classification", model="squeezebert/squeezebert-mnli-headless")
pipe2 = pipeline("zero-shot-classification", model="distilbert-base-uncased")

config.json:   0%|          | 0.00/500 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/202M [00:00<?, ?B/s]

Some weights of SqueezeBertForSequenceClassification were not initialized from the model checkpoint at squeezebert/squeezebert-mnli-headless and are newly initialized: ['classifier.bias', 'classifier.weight', 'transformer.pooler.dense.bias', 'transformer.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [ ]:
labels = ['Not Spam', 'Spam']
metric_results = []
prompts = ["corretly classicy the text with the given labels Not Spam and spam: ", "This is to test make it fail, just guess on the labels (FREE FREE FREE FREE FREE): "]

for prompt in tqdm(prompts):
    for text in tqdm(dataset['train'][:1000]['sms']):
        combined_text = prompt + text
        result1 = pipe(combined_text, labels)
        result2 = pipe2(combined_text, labels)
        metric_results.append({
            "question": prompt,
            "text": text,
            "predicted_label_pipe1": result1['labels'][0],
            "confidence_score_pipe1": result1['scores'][0],
            "predicted_label_pipe2": result2['labels'][0],
            "confidence_score_pipe2": result2['scores'][0]
        })



100%|██████████| 2/2 [16:46<00:00, 503.45s/it]


In [ ]:
print(metric_results)

[{'question': 'corretly classicy the text with the given labels Not Spam and spam: ', 'text': 'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...\n', 'predicted_label_pipe1': 'Spam', 'confidence_score_pipe1': 0.5016103982925415, 'predicted_label_pipe2': 'Spam', 'confidence_score_pipe2': 0.5003892779350281}, {'question': 'corretly classicy the text with the given labels Not Spam and spam: ', 'text': 'Ok lar... Joking wif u oni...\n', 'predicted_label_pipe1': 'Spam', 'confidence_score_pipe1': 0.5030966401100159, 'predicted_label_pipe2': 'Not Spam', 'confidence_score_pipe2': 0.500146746635437}, {'question': 'corretly classicy the text with the given labels Not Spam and spam: ', 'text': "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's\n", 'predicted_label_pipe1': 'Not Spam', 'confidence_score_pipe1': 0.5013719797134399, 'pr

In [18]:
def accuracy(true_labels, predicted_labels):
    correct = 0
    total = len(true_labels)
    for true_label, predicted_label in zip(true_labels, predicted_labels):
        if true_label == predicted_label:
            correct += 1
    return correct / total

In [29]:
ground_truth_labels = dataset['train'][:1000]['label']

#had chatgpt debug this for me

predicted_labels_pipe1_prompt1 = [result['predicted_label_pipe1'] for result in metric_results if result['question'] == prompts[0]]
predicted_labels_pipe2_prompt1 = [result['predicted_label_pipe2'] for result in metric_results if result['question'] == prompts[0]]
predicted_labels_pipe1_prompt2 = [result['predicted_label_pipe1'] for result in metric_results if result['question'] == prompts[1]]
predicted_labels_pipe2_prompt2 = [result['predicted_label_pipe2'] for result in metric_results if result['question'] == prompts[1]]

ground_truth_labels_ids = [id2label[label] for label in ground_truth_labels]

accuracy_pipe1_prompt1 = accuracy_score(ground_truth_labels_ids, predicted_labels_pipe1_prompt1)
accuracy_pipe2_prompt1 = accuracy_score(ground_truth_labels_ids, predicted_labels_pipe2_prompt1)
accuracy_pipe1_prompt2 = accuracy_score(ground_truth_labels_ids, predicted_labels_pipe1_prompt2)
accuracy_pipe2_prompt2 = accuracy_score(ground_truth_labels_ids, predicted_labels_pipe2_prompt2)

print("Accuracy for Pipeline 1 - Prompt 1:", 1 - accuracy_pipe1_prompt1)  # Inverted for classification task
print("Accuracy for Pipeline 2 - Prompt 1:", accuracy_pipe2_prompt1)
print("Accuracy for Pipeline 1 - Prompt 2:", 1 - accuracy_pipe1_prompt2)  # Inverted for classification task
print("Accuracy for Pipeline 2 - Prompt 2:", accuracy_pipe2_prompt2)

Accuracy for Pipeline 1 - Prompt 1: 0.782
Accuracy for Pipeline 2 - Prompt 1: 0.623
Accuracy for Pipeline 1 - Prompt 2: 0.751
Accuracy for Pipeline 2 - Prompt 2: 0.354


# **2.4 Baselines**


---


Next, compare these results to the type of model we have used in some of the
previous assignments. Use a library like scikit-learn to train a bag-of-words
classifier for your dataset. You may use tf-idf, word vectors, or anything else as
your input features, but you should not use any neural model that includes any
kind of recurrence or attention mechanism. You might use a logistic regression,
naive bayes, SVM, or anything else you think would be a reasonable baseline
model. Train the model on the training split and evaluate on the test split using
the same evaluation metrics that you did in the previous steps. This is your
BOW baseline.
Finally, add the following very simple baselines (which are always a good idea
to compare against, even if just as a sanity check to make sure your models are
learning something): random and majority/target-class. You do not actually
need to run any model to compute these – you just need to know the distribution
1However, we do have some free GCP credits available for students in this course; see
section 4. Note that you may want these for HW4 as well so try not to use all of your credits.
3
of labels in your test dataset. The random baseline is the theoretical score
achieved by a model making random guesses for every example. For example, if
the dataset contains two classes a random model will guess each class with equal
probability for each test example. If you do not wish to work out the scores
for this mathematically, you can also do it empirically by actually running a
large number of random samples (e.g., 10000) and recomputing the evaluation
metrics, then taking the average (mean) result. For the majority-class baseline,
you can use the same principles, but this is for a model that always predicts the
most common class, and this should be used if all classes are equally important.
For tasks using f1-score for only one class (e.g., like spam detection, where we
typically care about f1-score for the spam class only), you can use the target-class
baseline, where you always predict the “positive”, or target class (e.g., always
predict ’offensive’, or always predict ’spam’), even if it is not the most common
class. The idea is to select the most competitive baselines for your particular
task so that you can understand how much improvement you are gaining from
using your models. For example, if the majority class baseline score is .80 and
your best model achieves .81, this is much less impressive than if the majority
class baseline was .20. These are your random baseline and your majorityclass baseline (or target-class baseline if you are using target class f1-score
for evaluation).

In [30]:

dataset = load_dataset('sms_spam')
X_train, X_test, y_train, y_test = train_test_split(dataset['train']['sms'], dataset['train']['label'], test_size=0.2, random_state=22)

In [31]:
bow_model = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression())
])

#model listed above i got help from chat gpt on how to put all steps into pipeline

bow_model.fit(X_train,y_train)

bow_predictions = bow_model.predict(X_test)
bow_accuracy = accuracy_score(y_test, bow_predictions)

print("BOW Baseline Model Accuracy:", bow_accuracy)


BOW Baseline Model Accuracy: 0.9668161434977578
